# Intro

{Fill in with information about this notebook}

# Set Up notebook 

In [1]:
#Import modules
import numpy as np #Data manipulation
import pandas as pd #Point data manipulation and organization
import xarray as xr #Raster data manipulation and organization

import pathlib  #For filepaths, io, etc.
import os       #For several system-based commands
import datetime #For manipulation of time data, including file creation/modification times
import json     #For dictionary io, etc.

import matplotlib.pyplot as plt #For plotting and data vizualization
import geopandas as gpd         #For organization and manipulation of vector data in space (study area and some data points)
import rioxarray as rxr         #For orgnaization and manipulation of raster data
from scipy import interpolate
import shapely                  #For converting coordinates to point geometry
#Not sure if this cell is needed

In [2]:
import w4h

In [4]:
#Scripts with functions made for this specific application
import pathlib
import os
#Variables needed throughout, best to just assign now
todayDate, dateSuffix = w4h.get_current_date() 
repoDir = pathlib.Path(os.getcwd())

In [3]:
directoryDir = r'\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\\'[:-1]
downholeDataPATH, headerDataPATH, xyzInPATH  = w4h.filesSetup(db_dir=directoryDir)
headerDataIN, downholeDataIN = w4h.readRawTxtData(downholefile=downholeDataPATH, headerfile=headerDataPATH) #Functions to read data into dataframes. Also excludes extraneous columns, and drops header data with no location information
xyzDataIN = w4h.readXYZData(xyzfile=xyzInPATH)
downholeData = w4h.defineDataTypes(downholeDataIN, dtypeFile='downholeDataTypes.txt') #Define datatypes of each column of the new dataframes
headerData = w4h.defineDataTypes(headerDataIN, dtypeFile='headerDataTypes.txt')#Define datatypes of each column of the new dataframes
xyzData = w4h.defineDataTypes(xyzDataIN, dtypeFile='xyzDataTypes.txt')
studyAreaPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESL_StudyArea_5mi.shp"
studyAreaIN = w4h.read_study_area(studyAreaPath)
modelGridPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\grid_625_raster.tif"
surfaceElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ILStateLidar_ClipExtentESL.tif"
bedrockElevPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\SampleData\ESLBedrock.tif"
modelGrid = w4h.read_grid(datapath=modelGridPath, grid_type='model', studyArea=studyAreaIN,  read_grid=True, clip2SA=True)#, gridcrs='EPSG:26715', studyAreacrs='EPSG:26715')
surfaceElevGridIN = w4h.read_grid(datapath=surfaceElevPath, grid_type='surface', studyArea=studyAreaIN, use_service=False, clip2SA=True)
bedrockElevGridIN = w4h.read_grid(datapath=bedrockElevPath, grid_type='bedrock', studyArea=studyAreaIN, use_service=False, clip2SA=True)
#Code here for adding in control points
headerData = w4h.addElevtoHeader(xyzData, headerData) #This probably needs to be updated
headerData = w4h.coords2Geometry(df=headerData, xCol='LONGITUDE', yCol='LATITUDE', crs='EPSG:4269')
headerData = w4h.clipHeader2StudyArea(studyarea=studyAreaIN, headerdata=headerData, headerCRS='EPSG:4269')
downholeData = w4h.removeNonlocatedData(downholeData, headerData)
headerData = w4h.removenotopo(df=headerData, printouts=True)
donwholeData = w4h.dropnodepth(downholeData, printouts=True) #Drop records with no depth information
donwholeData = w4h.dropbaddepth(downholeData, printouts=True)#Drop records with bad depth information (i.e., top depth > bottom depth) (Also calculates thickness of each record)
downholeData = w4h.dropnoformation(downholeData, printouts=True)
downholeData.reset_index(inplace=True,drop=True) #These may not be necessary
headerData.reset_index(inplace=True,drop=True) #These may not be necessary
downholeData = pd.merge(left = downholeData, right = headerData, on='API_NUMBER')
specTermsPATH, startTermsPATH = w4h.searchTermFilePaths(dictdir=str(repoDir)+'/resources/', specStartPattern='*SearchTerms-Specific*', startGlobPattern = '*SearchTerms-Start*')
specTerms = w4h.read_dictionary_terms(dict_file=specTermsPATH)
startTerms = w4h.read_dictionary_terms(dict_file=startTermsPATH)
oldDictPath = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\WellData\Dictionaries\DICTIONARY_Updated-06-2018.csv"
oldDict = w4h.read_dictionary_terms(dict_file=oldDictPath, cols={'DESCRIPTION':'FORMATION', 'LITHOLOGY':'INTERPRETATION'}, class_flag=1)
specTerms = pd.concat([specTerms, oldDict])
specTerms.drop_duplicates(subset='FORMATION', inplace=True)
specTerms.reset_index(inplace=True, drop=True)
downholeData = w4h.specific_define(downholeData, specTerms, printouts=True)
classifedDF, searchDF = w4h.split_defined(downholeData)
searchDF = w4h.start_define(df=searchDF, starterms=startTerms, printouts=True)
downholeData = w4h.remerge_data(classifieddf=classifedDF, searchdf=searchDF)
classifedDF, searchDF = w4h.split_defined(downholeData)
searchDF = w4h.depth_define(searchDF, thresh=550, printouts=True)
downholeData = w4h.remerge_data(classifieddf=classifedDF, searchdf=searchDF)
downholeData = w4h.fill_unclassified(downholeData)
#dictDir = "\\\\isgs-sinkhole\\geophysics\\Balikian\\ISWS_HydroGeo\\WellDataAutoClassification\\SupportingDocs\\"
targetInterpDF = w4h.readLithologies()
downholeData = w4h.merge_lithologies(downholedata=downholeData, targinterps=targetInterpDF)
wellsDF = w4h.get_unique_wells(downholeData)
downholeData = w4h.sort_dataframe(df=downholeData, sort_cols=['API_NUMBER','TOP'], remove_nans=True)
inGrids = [bedrockElevGridIN, surfaceElevGridIN]
bedrockGrid, surfaceGrid = w4h.alignRasters(unalignedGrids=inGrids, modelgrid=modelGrid)
driftThickGrid, layerThickGrid = w4h.get_drift_thick(surface=surfaceGrid, bedrock=bedrockGrid, noLayers=9, plotData=False)
headerData = w4h.sample_raster_points(raster=bedrockGrid, ptDF=headerData, newColName='BEDROCK_ELEV_FT')
headerData = w4h.sample_raster_points(raster=surfaceGrid, ptDF=headerData, newColName='SURFACE_ELEV_FT')
headerData = w4h.sample_raster_points(raster=driftThickGrid, ptDF=headerData, newColName='BEDROCK_DEPTH_FT')
headerData = w4h.sample_raster_points(raster=layerThickGrid, ptDF=headerData, newColName='LAYER_THICK_FT')
headerData = w4h.get_layer_depths(well_metadata=headerData, no_layers=9)
downholeData_layerInfo = w4h.merge_tables(data_df=downholeData,  data_cols=None, header_cols=None, header_df=headerData,on='API_NUMBER', how='inner', auto_pick_cols=True)
#downholeData = downholeData_layerInfo.copy()
resdf = w4h.layer_target_thick(downholeData_layerInfo, layers=9, outfile_prefix='CoarseFine')
layers_data = w4h.layer_interp(points=resdf, layers=9, grid=modelGrid, method='lin')

Most Recent version of this file is : ISGS_DOWNHOLE_DATA_2023-01-06.txt
Most Recent version of this file is : ISGS_HEADER_2023-01-06.txt
Most Recent version of this file is : xyzData.csv
Using the following files:

\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_DOWNHOLE_DATA_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\ISGS_HEADER_2023-01-06.txt
\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\BedrockWellData\Wells\RawWellData_OracleDatabase\TxtData\xyzData.csv
Downhole Data has 3054409 valid well records.
Header Data has 636855 unique wells with valid location information.


c:\Users\riley\.conda\envs\raster38\lib\site-packages\geopandas\array.py:275: ShapelyDeprecationWarning: The array interface is deprecated and will no longer work in Shapely 2.0. Convert the '.coords' to a numpy array instead.
  return GeometryArray(vectorized.points_from_xy(x, y, z), crs=crs)


2998078 records removed without location information.
56331 wells remain from 7188 located wells in study area.
Well records removed: 0
Number of rows before dropping those without surface elevation information: 8150
Number of rows after dropping those without surface elevation information: 8150
Number of rows before dropping those without record depth information: 56331
Number of rows after dropping those without record depth information: 55747
Number of well records without formation information deleted: 584
Number of rows before dropping those with obviously bad depth information: 56331
Number of rows after dropping those with obviously bad depth information: 55725
Well records deleted: 606
Number of rows before dropping those without FORMATION information: 56331
Number of rows after dropping those without FORMATION information: 56331
Well records deleted: 0
Most Recent version of this file is : SearchTerms-Specific_2022-11-16_essCols.csv
Most Recent version of this file is : Search

c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\classify.py:57: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['CLASS_FLAG'].where(~df['FORMATION'].str.startswith(s,na=False),4,inplace=True)
c:\Users\riley\LocalData\Github\wells4hydrogeology\w4h\classify.py:58: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['INTERPRETATION'].where(~df['FORMATION'].str.startswith(s,na=False),starterms.loc[i,'INTERPRETATION'],inplace=True)


Records classified with start search term: 3586
Records classified with start search term: 14.49% of remaining data
Records classified as bedrock that were deeper than 550': 359
This represents 1.7% of the unclassified data in this dataframe.
Number of unique wells in downholeData: 7188
BEDROCK_ELEV_FT sampling should be done by 10:50
SURFACE_ELEV_FT sampling should be done by 10:50
BEDROCK_DEPTH_FT sampling should be done by 10:50
LAYER_THICK_FT sampling should be done by 10:50
REMOVING LATITUDE
REMOVING LONGITUDE
REMOVING ELEV_FT
REMOVING geometry
Index(['API_NUMBER', 'TABLE_NAME', 'FORMATION', 'THICKNESS', 'TOP', 'BOTTOM',
       'TOTAL_DEPTH', 'SECTION', 'TWP', 'TDIR', 'RNG', 'RDIR', 'MERIDIAN',
       'QUARTERS', 'ELEVATION', 'ELEVREF', 'COUNTY_CODE', 'ELEVSOURCE',
       'LATITUDE', 'LONGITUDE', 'ELEV_FT', 'geometry', 'INTERPRETATION',
       'CLASS_FLAG', 'BEDROCK_FLAG', 'TARGET', 'BEDROCK_ELEV_FT',
       'SURFACE_ELEV_FT', 'BEDROCK_DEPTH_FT', 'LAYER_THICK_FT',
       'LONGITUD

In [53]:
#Export (rio)xarray dataarrays and datasets
def export_grids(grid_data, out_path, file_id='',filetype='tif', variable_sep=True, date_stamp=True):
    """Function to export grids to files.

    Parameters
    ----------
    grid_data : xarray DataArray or xarray Dataset
        Dataset or dataarray to be exported
    out_path : str or pathlib.Path object
        Output location for data export. If variable_sep=True, this should be a directory. Otherwise, this should also include the filename. The file extension should not be included here.
    filetype : str, optional
        Output filetype. Can either be pickle or any file extension supported by rioxarray.rio.to_raster(). Can either include period or not., by default 'tif'
    variable_sep : bool, optional
        If grid_data is an xarray Dataset, this will export each variable in the dataset as a separate file, including the variable name in the filename, by default False
    date_stamp : bool, optional
        Whether to include a date stamp in the file name., by default True
    """

    #Initialize lists to determine which filetype will be used for export
    ncdfList = ['netcdf', 'ncdf', 'n']
    tifList = ['tif', 'tiff', 'geotiff', 'geotif', 't']
    pickleList = ['pickle', 'pkl', 'p']

    #Format output string(s)
    #Format output filepath
    if type(out_path) is str or isinstance(out_path, pathlib.PurePath):
        if isinstance(out_path, pathlib.PurePath):
            pass
        else:
            out_path = pathlib.Path(out_path)
        if out_path.parent.exists()==False:
            print('Directory does not exist. Please enter a different value for the out_path parameter.')
            return        

        if out_path.is_dir():
            if isinstance(grid_data, xr.DataArray):
                if variable_sep:
                    lyrs = grid_data.coords['Layer'].values
                    filenames = []
                    for l in lyrs:
                        filenames.append('Layer'+str(l))
                else:
                    filenames = ['AllLayers']
            if isinstance(grid_data, xr.Dataset):
                if variable_sep:
                    filenames = []
                    for var in grid_data:
                        filenames.append(var)
                else:
                    filenames = ['AllLayers']    
        else:
            filenames = [out_path.stem]
            out_path = out_path.parent

    else:
        print('Please input string or pathlib object for out_path parameters')
        return
    
    #Format datestamp, if desired in output filename
    if date_stamp:
        todayDate = datetime.date.today()
        todayDateStr = '_'+str(todayDate)
    else:
        todayDateStr=''

    #Ensure the file suffix includes .
    if filetype[0] == '.':
        pass
    else:
        filetype = '.' + filetype

    if file_id != '':
        file_id = '_'+file_id

    out_path = out_path.as_posix()+'/'
    outPaths = []
    for f in filenames:
        outPaths.append(out_path+f+file_id+todayDateStr+filetype)

    #Do export
    if filetype.lower() in pickleList:
        import pickle
        for op in outPaths:
            try:
                with open(op, 'wb') as f:
                    pickle.dump(grid_data, f)
            except:
                print('An error occured during export.')
                print(op, 'could not be exported as a pickle object.')
                print('Try again using different parameters.')
    else:
        import rioxarray as rxr
        try:
            if isinstance(grid_data, xr.Dataset):
                if variable_sep:
                    for i, var in enumerate(grid_data.data_vars):
                        grid_data[var].rio.to_raster(outPaths[i])
                else:
                    grid_data.rio.to_raster(outPaths[0])
            elif isinstance(grid_data, xr.DataArray):
                if variable_sep:
                    lyrs = grid_data.coords['Layer'].values
                    for i, l in enumerate(lyrs):
                        out_grid = grid_data.sel(Layer = l).copy()
                        out_grid.rio.to_raster(outPaths[i])
                else:
                    grid_data.rio.to_raster(outPaths[0])
            else:
                grid_data.rio.to_raster(outPaths[0])
        except:
            print('An error occured during export.')
            print('{} could not be exported as {} file.'.format(outPaths, filetype))
            print('Try again using different parameters.')

    return


In [46]:
out_dir = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\ProcessedData"
out_dir = pathlib.Path(out_dir)
out_dir.parent

WindowsPath('//isgs-sinkhole.ad.uillinois.edu/geophysics/Balikian/ISWS_HydroGeo/WellDataAutoClassification')

In [54]:
out_dir = r"\\isgs-sinkhole.ad.uillinois.edu\geophysics\Balikian\ISWS_HydroGeo\WellDataAutoClassification\ProcessedData"
export_grids(grid_data=layers_data, out_path=out_dir, file_id='Coarse', filetype='tif', variable_sep=True, date_stamp=True)